In [2]:
from music21 import *

In [3]:
def find_Z_relation(sc):
    """Given a score object, identifies and labels where Z relations occur in the piece."""
    pass

# Helper Functions

### - Normal Form

In [17]:
def perm_list(pitch_list, rotation_amount=1):
    """Rotates a list by rotation_amount"""
    return pitch_list[rotation_amount:] + pitch_list[:rotation_amount]

In [19]:
def check_compactness(pitch_classes, index = -1):
    """Checks distance between first and last notes"""
    return (pitch_classes[index] - pitch_classes[0]) % 12

In [29]:
def check_equal(list1, list2, index):
    """Checks equality if there is a tie for the best intervals between pitches when finding the normal form"""
    if index == -len(list1): # if the lists have equal pitches classes all the way down. Take the one with the lowest starting pitch class
        if list1[0] < list2[0]:
            return list1
        return list2
    if check_compactness(list1, index) == check_compactness(list2, index):
        return check_equal(list1, list2, index-1)
    elif check_compactness(list1, index) < check_compactness(list2, index):
        return list1
    return list2

In [81]:
def most_compact(pitches):
    best_interval, best_pitches = check_compactness(pitches), pitches
    for i in range(len(pitches)):
        rotation = perm_list(pitches, i)
        compactness = check_compactness(rotation)
        if compactness < best_interval:
            best_interval = compactness
            best_pitches = rotation
        if compactness == best_interval:
            best_pitches = check_equal(best_pitches, rotation, index=-2)
    return best_pitches

In [82]:
def reduce_to_normal_form(pitch_list):
    """
    Given a list of musci21 pitch objects, reduces them to their normal form as a 
    list of integers.
    """
    pitch_classes = sorted(list(set(x.pitchClass for x in pitch_list)))
    normal_form = most_compact(pitch_classes)
    return normal_form


In [83]:
# Tests for normal form
def test_normal_form():
    """Runs test cases for the reduction to normal form"""
    P = pitch.Pitch
    pc_ex = [P('C4'), P('C#4'), P('C3'), P('B3')]
    pc1 = [P('Ab4'), P('D4'),P('Eb5'), P('A4')]
    pc2 = [P('B5'), P('G4'), P('D5'), P('Eb'), P('D6')]
    assert reduce_to_normal_form(pc1) == [2,3,8,9]
    assert reduce_to_normal_form(pc_ex) == [11,0,1]
    assert reduce_to_normal_form(pc2) == [11,2,3,7]
    pass
test_normal_form()

### - Prime Form


In [85]:
def transpose_normal(pitch_classes):
    """Given a list of pitch classes in normal form, transposes them to start on C."""
    return [(x+12 - pitch_classes[0])%12 for x in pitch_classes]


In [87]:
def invert_transpose(pitch_classes):
    """Given a list of pitch classes in normal form, transposes them to start on C, then inverts 
    them, then put the inverted form into increasing order."""
    transposed = [(x+12 - pitch_classes[0])%12 for x in pitch_classes]
    inverted = [(-x)%12 for x in transposed]
    increasing = perm_list(sorted(inverted), 1)
    return increasing, transposed

In [73]:
lis = [0,3,4,8]
invert_transpose([0,3,4,8])

[4, 8, 9, 0]

In [143]:
def reduce_to_prime_form(pitch_list, stringify = False):
    """reduces a pitch list to prime form. If stringify = True, then returns the answer as a string. """
    
    inverted, transposed_normal  = invert_transpose(pitch_list)
    best_inverted = most_compact(inverted)
    if check_compactness(best_inverted,) == check_compactness(transposed_normal):
        anslist = check_equal(best_inverted, transposed_normal, index=-2)
    elif check_compactness(best_inverted,) < check_compactness(transposed_normal):
        anslist = best_inverted
    elif check_compactness(best_inverted,) > check_compactness(transposed_normal):
        anslist =  transposed_normal
    transpose2 = transpose_normal(anslist)
    if not stringify:
        return transpose2
    prime_form = ''
    for x in transpose2:
        if x ==10:
            x = 'T'
        elif x == 11:
            x = 'E'
        prime_form = prime_form + str(x)
    return prime_form

    

In [147]:
def test_prime_form():
    """Tests the function reduce_to_prime_form"""
    assert reduce_to_prime_form([11,2,3,7], stringify=True) == '0148'
    assert reduce_to_prime_form([11,2,3,7]) == [0,1,4,8]
    pass


In [148]:
#TODO: add more test cases
test_prime_form()

### - Interval Vectors

In [149]:
def get_interval_vector(pitch_list, zero = False):
    """Given a list of pitches, returns the interval vector between all of the pitches. 
    If zero = true, returns the interval vector that includes 0.
    """
    intervals = {}
    for i in range(len(pitch_list)):
        for j in range(len(pitch_list)-i):
            diff = pitch_list[j+i]-pitch_list[i]
            if diff >6:
                diff = 12-diff
            if diff in intervals:
                intervals[diff] +=1
            else:
                intervals[diff] = 1
    interval_vector = []
    for i in range(7):
        if i in intervals:
            interval_vector.append(intervals[i])
        else:
            interval_vector.append(0)
    if zero:
        return interval_vector
    return interval_vector[1:]

In [156]:
def test_interval_vector():
    a = reduce_to_prime_form([11,2,3,7], stringify=False)
    P = pitch.Pitch
    g_half_dim = [P('G4'), P('Bb4'), P('Db5'), P('F5') ]
    G_normal = reduce_to_normal_form(g_half_dim)
    G_prime = reduce_to_prime_form(G_normal)
    assert get_interval_vector(G_prime) == [0,1,2,1,1,1]
    assert get_interval_vector(a) == get_interval_vector(a)
test_interval_vector()